# IBM Instance Setup

In [1]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [2]:
NODES = 14
SP = "Sparse"
MD = "Medium"
DS = "Dense"


ER = "ER"
BP = "Bipartite"
CM = "Complete"

In [3]:
ER14NSParams = np.load("results/params/params_14N_ER_Sparse_LR0.000674683777427142_SIGMA0.7311700596180342_INIT0.3696050646549004_NUMLAYERS5.npy")
ER14NMParams = np.load("results/params/params_14N_ER_Medium_LR0.020808717745815757_SIGMA0.8897430791197918_INIT0.6417265067371234_NUMLAYERS3.npy")
ER14NDParams = np.load("results/params/params_14N_ER_Dense_LR0.003007928492112876_SIGMA0.788257573897633_INIT0.45428405466863664_NUMLAYERS2.npy")
BP14NSParams = np.load("results/params/params_14N_Bipartite_Sparse_LR0.001672169486739108_SIGMA0.9231013598757258_INIT0.2562470060605544_NUMLAYERS5.npy")
BP14NMParams = np.load("results/params/params_14N_Bipartite_Medium_LR0.003700878888183011_SIGMA1.8896275887587226_INIT1.4096892767484017_NUMLAYERS4.npy")
BP14NDParams = np.load("results/params/params_14N_Bipartite_Dense_LR0.05697641941280111_SIGMA0.10993054267369362_INIT1.332910357138958_NUMLAYERS1.npy")

# Hardware test

In [4]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=156)
print(backend)

<IBMBackend('ibm_aachen')>


In [5]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=512,
                    optimization_level=3)

## Experiments

In [6]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()

L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [7]:
with Session(backend=backend, max_time="12h") as session:
    ER14NSRes  = L2Sample(ER14NSParams)
    ER14NMRes  = L3Sample(ER14NMParams)
    ER14NDRes  = L5Sample(ER14NDParams)

    BP14NDRes = L1Sample(BP14NDParams)
    BP14NMRes = L4Sample(BP14NMParams)
    BP14NSRes = L5Sample(BP14NSParams)

In [8]:
np.save("results/samples/2025-06-07/ER14NS.npy", ER14NSRes)
np.save("results/samples/2025-06-07/ER14NM.npy", ER14NMRes)
np.save("results/samples/2025-06-07/ER14ND.npy", ER14NDRes)
np.save("results/samples/2025-06-07/BP14NS.npy", BP14NSRes)
np.save("results/samples/2025-06-07/BP14NM.npy", BP14NMRes)
np.save("results/samples/2025-06-07/BP14ND.npy", BP14NDRes)
print("Samples saved successfully.")

Samples saved successfully.
